In [1]:
import numpy as np
import pandas as pd
import re
from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.tokenizer import tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from collections import Counter
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [2]:
# read the dataset
reviews_data = pd.read_json(r"C:\Users\user\Documents\UoM\Thesis\data\Clothing_Shoes_and_Jewelry_5.json", lines=True)

# Keep only the review text and the grade
reviews_data = reviews_data[['overall', 'reviewText']]


# Drop the products whose values are null
reviews_data = reviews_data[reviews_data['reviewText'].notna()]

In [3]:
ratings = []
for index,entry in enumerate(reviews_data['overall']):
    if entry == 1.0 or entry == 2.0:
        ratings.append(-1)
    elif entry == 3.0:
        ratings.append(0)
    elif entry == 4.0 or entry == 5.0:
        ratings.append(1)

In [4]:
""""
Split each review into sentences
and preprocess each sentence
"""
stopwords = stopwords.words('english')
lc = LancasterStemmer()

preprocessed_data = []
index = 0
for review in reviews_data['reviewText']:
    if(index % 10000 == 0):
        print(index)
    review_sentences = tokenizer.tokenize(review)
    for sentence in review_sentences:
        sentence = re.sub('[^a-zA-Z]', ' ', sentence)
        sentence = sentence.lower()
        sentence = word_tokenize(sentence)
        sentence = [lc.stem(word) for word in sentence if not word in stopwords]
        sentence = ' '.join(sentence)
        if(sentence != ''):
            review = {}
            review["index"] = index
            review["sentence"] = sentence
            review["rating"] = ratings[index]
            preprocessed_data.append(review)
    index += 1

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000


In [5]:
print("Before preprocessing: ", reviews_data['reviewText'][278643])
print("------------------------------------------------")
print("After preprocessing: ", [d for d in preprocessed_data if d['index'] == 278643])
print(len(preprocessed_data))

Before preprocessing:  idk why this shirt didnt fit i mean i ordered an Xlits very short and it seems like it was cut weird because the cross sits funny, like sideways.also one of the studs fell off the second i tried it on. ahhhhhhh whatever thats what i get for ordering clothes online loli DO NOT recommend
------------------------------------------------
After preprocessing:  [{'index': 278643, 'sentence': 'idk shirt didnt fit mean ord xlit short seem lik cut weird cross sit funny lik sideway also on stud fel second tri', 'rating': -1}, {'index': 278643, 'sentence': 'ahhhhhhh whatev that get ord cloth onlin lol recommend', 'rating': -1}]
1172750


In [6]:
# Split Dataset into training and test
indexes = [i for i in range(len(reviews_data))]

Train_X_index, Test_X_index, Train_Y_review, Test_Y_review = model_selection.train_test_split(indexes,ratings,test_size=0.3)

df = pd.DataFrame(preprocessed_data)

train = df[df['index'].isin(Train_X_index)]
Train_Y = train['rating'].tolist()
Train_X = train['sentence'].tolist()
Train_index = train['index'].tolist()


test = df[df['index'].isin(Test_X_index)]
Test_Y = test['rating'].tolist()
Test_X = test['sentence'].tolist()
Test_index = test['index'].tolist()

all_sentences = Train_X + Test_X

In [7]:
len(df)

1172750

In [8]:
# Encoding of label
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
Train_Y_review = Encoder.fit_transform(Train_Y_review)
Test_Y_review = Encoder.fit_transform(Test_Y_review)

In [9]:
# Word Vectorization
        
TfIdf_vect = TfidfVectorizer(max_features=30000)
TfIdf_vect.fit(all_sentences)
Train_X_TfIdf = TfIdf_vect.transform(Train_X)
Test_X_TfIdf = TfIdf_vect.transform(Test_X)

In [10]:
def find_Max_Element(scores):
    return  max(set(scores), key = scores.count)

In [11]:
# Support Vector Machine with linear cernel

from sklearn.svm import LinearSVC
import timeit

start = timeit.default_timer()

# Train the classifier
clf = LinearSVC(random_state=5, tol=1e-5)
clf.fit(Train_X_TfIdf,Train_Y)

stop = timeit.default_timer()

print("Training is done. Time: ", (stop-start)) 

predictions_SVM = clf.predict(Test_X_TfIdf)

print("Predicting is done")

# Get grades by review
predicted_scores_frame = pd.DataFrame(np.column_stack([Test_index, predictions_SVM, Test_Y]), 
                               columns=['review Index', 'prediction', 'actual'])

print(len(predicted_scores_frame['review Index'].unique()) - len(Test_Y_review))

# for each review get all scores by review
scores_by_review_frame = predicted_scores_frame.groupby('review Index')['prediction'].apply(list)
majority_scores =  scores_by_review_frame.apply(find_Max_Element)
predicted_scores = list(majority_scores)

# for each review get its actual score
actual_scores_frame = predicted_scores_frame.groupby('review Index')['actual'].first()
actual_scores = list(actual_scores_frame)

# Calculate Accuracy
accuracy = accuracy_score(predicted_scores, actual_scores)*100
print("Accuracy: ", accuracy)

# Confusion Matrix
cm = confusion_matrix(actual_scores, predicted_scores)
print("Confusion matrix: ", cm)

# Classification Report
my_tags = ['Negative','Neutral','Positive']
report = classification_report(actual_scores, predicted_scores, target_names=my_tags);

Training is done. Time:  1027.0498193999997
Predicting is done
-5
Accuracy:  80.45072309477386
Confusion matrix:  [[  936    42  7030]
 [  287   135  8716]
 [  191    77 66185]]
